In [13]:
#!pip install openai
import openai
import pandas as pd
import os
import time
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']  # input yours here

In [48]:
df = pd.read_csv("data/data1.csv")
df.shape

(20, 10)

In [49]:
df.head()

,submission_id,title,text,num_votes,num_comments,date,author,author_id,author_name,comments
0,1akynbd,L'UE recule sur la réduction des pesticides : ...,NaN,70,25,"07/02/2024, 08:52:31",t2_t16fr,t16fr,Folivao,"[""Oh ben ça alors. Au moins l'agrobusiness va ..."
1,1aleoht,Si vous deviez noter l'utilité réel de votre t...,"0 étant le bulshit job ultime, genre ""consulta...",222,651,"07/02/2024, 21:48:23",t2_dfxglyvc0,dfxglyvc0,ad-undeterminam,"[""1/10. Je bosse au service info (architecte /..."
2,1agx03j,Gabriel Attal tente d’éteindre la colère des a...,NaN,264,150,"02/02/2024, 06:12:57",t2_2i152lwx,2i152lwx,Proudhmak87,"['Sans blague ? Je suis très étonné.', 'Ce qui..."
3,1agk0hy,Prague veut une réponse forte face aux agricul...,NaN,166,25,"01/02/2024, 19:58:30",t2_5bnih6r,5bnih6r,DicentricChromosome,"[""M. Tchekov, en tant que ministre de l'intéri..."
4,1ael0kl,Pourquoi les agriculteurs n'achètent pas franç...,Une des revendications de la manif des agri es...,104,403,"30/01/2024, 10:04:38",t2_3ooog9vx,3ooog9vx,papuniu,"['Je ne reconnais plus personne, en Massey Fer..."


In [63]:
df = df[['title','comments','date']]

In [131]:
llm_model = "gpt-3.5-turbo"
prompt = """\
Je vais te donner une liste d'information avec un titre et la liste de ses commentaires,
Tu vas faire du sentimal analysis sur les données fournies
Tu vas t'interesser au titre et à la liste des commentaires correspondant à ce titre.
Tu vas faire du sentimental analysis pour chaque commentaire de ce titre.
Sentiment: stockera l'analyse que tu as fait du commentaire. 
title :  le titre du post
comments: liste des commentaires
sentiment: Extraire le sentiment du texte et le restituer sous forme négative, positive ou neutre.
argument: Extraire l'argument derrière le sentiment et le restituer sous la forme d'une liste de Pyhton.
Tu retourneras une liste contenant le titre, les commentaires et le sentiment pour chaque commentaire.
Retourne juste un objet json

Format de la réponse: une liste JSON avec les clés suivantes:
title
comments
sentiment
argument

liste d'information: {data}
"""

In [132]:
def get_response_from_model(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=llm_model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [133]:
liste_data = df[['title','comments']]

In [134]:
print(prompt.format(data=list(liste_data.iloc[0])))

Je vais te donner une liste d'information avec un titre et la liste de ses commentaires,
Tu vas faire du sentimal analysis sur les données fournies
Tu vas t'interesser au titre et à la liste des commentaires correspondant à ce titre.
Tu vas faire du sentimental analysis pour chaque commentaire de ce titre.
Sentiment: stockera l'analyse que tu as fait du commentaire. 
title :  le titre du post
comments: liste des commentaires
sentiment: Extraire le sentiment du texte et le restituer sous forme négative, positive ou neutre.
argument: Extraire l'argument derrière le sentiment et le restituer sous la forme d'une liste de Pyhton.
Tu retourneras une liste contenant le titre, les commentaires et le sentiment pour chaque commentaire.
Retourne juste un objet json

Format de la réponse: une liste JSON avec les clés suivantes:
title
comments
sentiment
argument

liste d'information: ["L'UE recule sur la réduction des pesticides : cadeau aux agriculteurs ou à l'agrochimie ?", '["Oh ben ça alors. Au

In [135]:
response = get_response_from_model(prompt.format(data=liste_data))

In [122]:
print(response)

[
  {
    "title": "L'UE recule sur la réduction des pesticides : les agriculteurs en colère",
    "comments": [
      "Oh ben ça alors. Au moins l'agrobusiness va pouvoir continuer à empoisonner les sols et les nappes phréatiques.",
      "1/10. Je bosse au service info (architecte / devops) d'une grosse boîte de l'agroalimentaire. Je peux vous dire que les agriculteurs sont les premiers à utiliser des pesticides à tout va. Ils sont les premiers à demander des produits chimiques pour augmenter leurs rendements. Ils sont les premiers à utiliser des OGM. Ils sont les premiers à utiliser des semences hybrides. Ils sont les premiers à utiliser des engrais chimiques. Ils sont les premiers à utiliser des antibiotiques pour leurs animaux. Ils sont les premiers à utiliser des hormones pour leurs animaux. Ils sont les premiers à utiliser des antibiotiques pour leurs animaux. Ils sont les premiers à utiliser des hormones pour leurs animaux. Ils sont les premiers à utiliser des antibiotiques pou

In [115]:
print(response)

{
  "title": "L'UE recule sur la réduction des pesticides : les agriculteurs en colère",
  "comments": [
    {
      "comment": "Oh ben ça alors. Au moins l'agrobusiness va pouvoir continuer à empoisonner les sols et les nappes phréatiques en toute impunité.",
      "sentiment": "négatif",
      "argument": ["empoisonnement des sols", "empoisonnement des nappes phréatiques"]
    },
    {
      "comment": "1/10. Je bosse au service info (architecte / devops) d'une grosse boîte de l'agroalimentaire. Je peux vous dire que les agriculteurs sont les derniers de nos soucis. On préfère investir dans des projets de R&D pour améliorer nos produits et nos processus.",
      "sentiment": "négatif",
      "argument": ["manque d'intérêt pour les agriculteurs", "priorité à la R&D"]
    },
    {
      "comment": "Sans blague ? Je suis très étonné.",
      "sentiment": "neutre",
      "argument": []
    },
    {
      "comment": "M. Tchekov, en tant que ministre de l'intérieur, vous devriez vous occup

In [102]:
import json
r = json.loads(response)
print(type(r))

<class 'dict'>


In [107]:
r['title']

"L'UE recule sur la réduction des pesticides : les agriculteurs en colère"

In [110]:
comment_data = []
for comment in r['comments']:
    comment_data.append({
        'title': r['title'],
        'comment': comment['comment'],
        'sentiment': comment['sentiment']
    })

# Créer un DataFrame à partir de la liste de dictionnaires
df2 = pd.DataFrame(comment_data)

In [114]:
r['comments'][0]

{'comment': "Oh ben ça alors. Au moins l'agrobusiness va pouvoir continuer à empoisonner les sols et les nappes phréatiques en toute impunité.",
 'sentiment': 'négatif',
 'argument': ['empoisonnement des sols',
  'empoisonnement des nappes phréatiques']}

In [111]:
df2

,title,comment,sentiment
0,L'UE recule sur la réduction des pesticides : ...,Oh ben ça alors. Au moins l'agrobusiness va po...,négatif
1,L'UE recule sur la réduction des pesticides : ...,1/10. Je bosse au service info (architecte / d...,négatif
2,L'UE recule sur la réduction des pesticides : ...,Sans blague ? Je suis très étonné.,neutre
3,L'UE recule sur la réduction des pesticides : ...,"M. Tchekov, en tant que ministre de l'intérieu...",négatif
4,L'UE recule sur la réduction des pesticides : ...,"Je ne reconnais plus personne, en Massey Fergu...",neutre
5,L'UE recule sur la réduction des pesticides : ...,Les agriculteurs ont tout mon soutien. Ils son...,positif
6,L'UE recule sur la réduction des pesticides : ...,La manière dont ils cèdent à toutes les revend...,négatif
7,L'UE recule sur la réduction des pesticides : ...,Rarement vu un article *ragebait* aussi peu su...,négatif
8,L'UE recule sur la réduction des pesticides : ...,Je vous demande de vous arrêter,négatif
9,L'UE recule sur la réduction des pesticides : ...,Ah? C'est pas déjà fait?,neutre


In [99]:
new_df= pd.DataFrame(columns=('title','comments','sentiment'))


In [2]:
llm_model = "gpt-3.5-turbo"
prompt = """\
For the following text, extract the following information:
item :  what is the purchased item?
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
gift for : If it is a gift  say for who? otherwise output -1
delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
sentiment: Extrcat the sentiment from the text and output as negative, positive or neutral 
argument: Extract the argument behind the sentiment and output as Pyhton list.

Format the output as JSON with the following keys:
item
gift
gift for
delivery_days
price_value
sentiment
argument

text: {text}
"""
def get_response_from_model(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=llm_model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [3]:
review = "I purchased a table very bad experience 2 days delay to receive my item"

In [4]:
prompt.format(text=review)

'For the following text, extract the following information:\nitem :  what is the purchased item?\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\ngift for : If it is a gift  say for who? otherwise output -1\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\nsentiment: Extrcat the sentiment from the text and output as negative, positive or neutral \nargument: Extract the argument behind the sentiment and output as Pyhton list.\n\nFormat the output as JSON with the following keys:\nitem\ngift\ngift for\ndelivery_days\nprice_value\nsentiment\nargument\n\ntext: I purchased a table very bad experience 2 days delay to receive my item\n'

In [5]:
print(get_response_from_model(prompt.format(text=review)))

{
  "item": "table",
  "gift": false,
  "gift for": -1,
  "delivery_days": 2,
  "price_value": [],
  "sentiment": "negative",
  "argument": ["very bad experience"]
}
